<a href="https://colab.research.google.com/github/Ananda0047/Bangla-Sentiment-Analysis/blob/main/Inter_Annotator_Agreement_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Bangla_Sent_zahin.xlsx to Bangla_Sent_zahin.xlsx
Saving Bangla_Sent_wasay.xlsx to Bangla_Sent_wasay.xlsx


In [ ]:
import io
dfw = pd.read_excel(io.BytesIO(uploaded['Bangla_Sent_wasay.xlsx']))
# Dataset is now stored in a Pandas Dataframe

import io
dfz = pd.read_excel(io.BytesIO(uploaded['Bangla_Sent_zahin.xlsx']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
def cohen_kappa(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations provided by first annotator
    :type ann1: list
    :param ann2: annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

In [ ]:
# Merge the DataFrames column-wise based on the "text" column
dfwz = pd.merge(dfw, dfz, on='text', how='outer')

# Rename the columns to match the desired naming convention
dfwz = dfwz.rename(columns={'Sentiment_x': 'sentiment_w', 'Sentiment_y': 'sentiment_z'})

In [ ]:
dfwz.isnull().sum()

text           0
sentiment_w    9
sentiment_z    1
dtype: int64

In [ ]:
# Drop rows with null values
dfwz = dfwz.dropna()

In [ ]:
dfwz

,text,sentiment_w,sentiment_z
0,অই জায়গার ডাক্তাররা কি গু খায় কে পাগল কে সুস্থ...,disgust,disgust
1,অনেকদিন হলো তাকে দেখিনা মাঝে মাঝে কান্না আসে,sadness,sadness
2,অগণতান্ত্রিক একটি জঘন্য দেশে আমার বাস,disgust,disgust
3,অটো রিস্কা বন্ধ করতে হবে তা না হলে আরো সংকট বে...,fear,fear
4,অটোরিকশা বন্ধ না করলে আরও সংকট হবে মনে হচ্ছে,fear,fear
...,...,...,...
6755,পদ্মা সেতু থাকেলই হবে কারেন্ট দিয়া কি হবে,'Doesn't fall under any category','Doesn't fall under any category'
6756,ধান বেচে পায়খানা বানালুম হাগুম কি খায়া,'Doesn't fall under any category','Doesn't fall under any category'
6757,এখানে কমেন্ট করা লোকগুলোর মধ্যে কি কোনো আওয়ামী...,'Doesn't fall under any category','Doesn't fall under any category'
6758,উন্নয়ন উন্নয়ন উন্নয়ন উন্নয়ন টাইম প্রতিদিন ...,'Doesn't fall under any category','Doesn't fall under any category'


In [ ]:
cohen_kappa(dfwz["sentiment_w"].to_list(), dfwz["sentiment_z"].to_list())

0.7227

In [ ]:
# Delete rows with specific values in the "Sentiment" column
dfwz_filtered = dfwz[~((dfwz['sentiment_w'].isin(["'Doesn't fall under any category'", 'Not sure'])))]
dfwz_filtered = dfwz_filtered[~((dfwz_filtered['sentiment_z'].isin(["'Doesn't fall under any category'", 'Not sure'])))]

In [ ]:
dfwz_filtered

,text,sentiment_w,sentiment_z
0,অই জায়গার ডাক্তাররা কি গু খায় কে পাগল কে সুস্থ...,disgust,disgust
1,অনেকদিন হলো তাকে দেখিনা মাঝে মাঝে কান্না আসে,sadness,sadness
2,অগণতান্ত্রিক একটি জঘন্য দেশে আমার বাস,disgust,disgust
3,অটো রিস্কা বন্ধ করতে হবে তা না হলে আরো সংকট বে...,fear,fear
4,অটোরিকশা বন্ধ না করলে আরও সংকট হবে মনে হচ্ছে,fear,fear
...,...,...,...
6429,আমাদের হবিগঞ্জ জেলা মাধবপুর উপজেলার বিদ্যুৎ থা...,anger,sadness
6431,এজন্য দায়ি জনগণ কারন যেখানে বাল্ব লাগবে একটি আ...,sadness,anger
6432,উন্নয়নের‌ ঠেলায় ‌এখন কারেন্ট থাকে না,anger,disgust
6435,উন্নত দেশে লোডশেডিংয়,surprise,sadness


In [ ]:
cohen_kappa(dfwz_filtered["sentiment_w"].to_list(), dfwz_filtered["sentiment_z"].to_list())

0.8593

In [ ]:
# Keep rows where sentiment_w and sentiment_z have the same values
Bangla_Sentiment = dfwz_filtered[dfwz_filtered['sentiment_w'] == dfwz_filtered['sentiment_z']]

In [ ]:
Bangla_Sentiment

,text,sentiment_w,sentiment_z
0,অই জায়গার ডাক্তাররা কি গু খায় কে পাগল কে সুস্থ...,disgust,disgust
1,অনেকদিন হলো তাকে দেখিনা মাঝে মাঝে কান্না আসে,sadness,sadness
2,অগণতান্ত্রিক একটি জঘন্য দেশে আমার বাস,disgust,disgust
3,অটো রিস্কা বন্ধ করতে হবে তা না হলে আরো সংকট বে...,fear,fear
4,অটোরিকশা বন্ধ না করলে আরও সংকট হবে মনে হচ্ছে,fear,fear
...,...,...,...
4816,হোগার পোলারা কি শুরু করছোস এই তদের ডিজিটাল দেশ...,anger,anger
4817,হ্যাঁ আপনার কথা এবং যু‌ক্তির সা‌থে একমত,joy,joy
4818,হ্যাঁ এটাই ঠিক,joy,joy
4819,হ্যাঁ বদলে গেছি সময়ের সাথে তাল মিলিয়ে অনুভূতির...,sadness,sadness


In [ ]:
Bangla_Sentiment.drop('sentiment_z', axis=1)

,text,sentiment_w
0,অই জায়গার ডাক্তাররা কি গু খায় কে পাগল কে সুস্থ...,disgust
1,অনেকদিন হলো তাকে দেখিনা মাঝে মাঝে কান্না আসে,sadness
2,অগণতান্ত্রিক একটি জঘন্য দেশে আমার বাস,disgust
3,অটো রিস্কা বন্ধ করতে হবে তা না হলে আরো সংকট বে...,fear
4,অটোরিকশা বন্ধ না করলে আরও সংকট হবে মনে হচ্ছে,fear
...,...,...
4816,হোগার পোলারা কি শুরু করছোস এই তদের ডিজিটাল দেশ...,anger
4817,হ্যাঁ আপনার কথা এবং যু‌ক্তির সা‌থে একমত,joy
4818,হ্যাঁ এটাই ঠিক,joy
4819,হ্যাঁ বদলে গেছি সময়ের সাথে তাল মিলিয়ে অনুভূতির...,sadness


In [ ]:
Bangla_Sentiment = Bangla_Sentiment.rename(columns={'text': 'Text', 'Sentiment_w': 'Sentiment'})